In [1]:
from openai import OpenAI
import pandas as pd

with open('../api_key.txt', 'r') as file:
            key = file.read().strip()

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=key,
)

In [2]:
data = pd.read_csv("../data/1867_austria.csv").drop('Unnamed: 0', axis=1)

a_null_entry = data.iloc[314,:]
an_almost_correct_entry = data.iloc[639, :] # Milau
a_wrong_entry = data.iloc[60, :] # Should be: Lugo de Vicence, is: lamain
a_second_wrong_entry = data.iloc[1572, :]

entries = [a_null_entry, an_almost_correct_entry, a_wrong_entry, a_second_wrong_entry]

data

,text,source,class,location
0,PRODUITS D'IMPRIMERIE ET DE LIBRAIRIC. 2] EMPI...,Paris1867_Austria_0.jpg,NaN,NaN
1,"1. Beek (maison Alfred Hülder), àVienne. — Liv...",Paris1867_Austria_0.jpg,CLASSE 6 — PRODUITS D'IMPRIMERIE\n: ET DE LIBR...,Vienne
2,"3 Braumäller (Guillaume), àVienne. — Livres de...",Paris1867_Austria_0.jpg,CLASSE 6 — PRODUITS D'IMPRIMERIE\n: ET DE LIBR...,Vienne
3,"3. Caumo (Antoine), à Rovéredo(Tyrol). — Epreu...",Paris1867_Austria_0.jpg,CLASSE 6 — PRODUITS D'IMPRIMERIE\n: ET DE LIBR...,Rovéredo
4,4. Corporation des libraireset des marchands d...,Paris1867_Austria_0.jpg,CLASSE 6 — PRODUITS D'IMPRIMERIE\n: ET DE LIBR...,vienne
...,...,...,...,...
3464,"6. Krzcptowski (Joseph), à Koscieiisko (Gallic...",Paris1867_Austria_90.jpg,CLASSE 94. — PRODUITS DR TOUTE\nSORTE FABRIQUÉ...,Koscieiisko
3465,"7. Laporzynski (Romuald), à Neumarkt (Gallicie...",Paris1867_Austria_90.jpg,CLASSE 94. — PRODUITS DR TOUTE\nSORTE FABRIQUÉ...,Neumarkt
3466,"8. Lay (Félix), à Essegg (Esclavone — Tapis, f...",Paris1867_Austria_90.jpg,CLASSE 94. — PRODUITS DR TOUTE\nSORTE FABRIQUÉ...,Essegg
3467,4er réen de la frontière de Lika:— Couverture ...,Paris1867_Austria_90.jpg,CLASSE 94. — PRODUITS DR TOUTE\nSORTE FABRIQUÉ...,NaN


In [3]:
def query_chatgpt(query):
    chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
    model="gpt-3.5-turbo",)
    
    return(chat_completion.choices[0].message.content)

In [4]:
question = """
In the following string, which contains potential typos, I am looking for a city name.
I know the city is located in the former Austro-Hungarian Empire so please mention a city in that area. 
Give me the name of the city, no other output:"
"""

question_without_breaks = question.replace('\n', '')

In [20]:
responses_from_3243 = []

for i in data['text'][3243:]:
    query = question_without_breaks + " " + i
    responses_from_3243.append(query_chatgpt(query))


In [21]:
#data['responses'] = responses
responses_from_3243

['Vienna',
 'Opava',
 'The city name in the given string is Troppau.',
 'Leoben',
 'Kaschau',
 'Debreczin',
 'The city mentioned in the string is "CastelNuovo de Frau".',
 'The city name in the given string is "Nünchengratz".',
 'The city name mentioned in the given string is Prague.',
 'The city mentioned in the string is "Weisskirchen".',
 'I cannot find a city name in the given string.',
 'The city mentioned in the string is Vienna (Vienne).',
 'Ujhely',
 'The city name mentioned in the given string is "Kolin".',
 'The city name mentioned in the string is "Windisgrâtz".',
 'Marburg (Styrie)',
 'Vienna',
 'The city name in the given string is not clear due to the presence of potential typos. Could you please provide another string where the city name is more identifiable?',
 'The city mentioned in the given string is "Mahresh Weesskirchen".',
 'Vienna',
 'The city name you are looking for is Canjevo in Croatia.',
 'Trieste',
 'Dioszegh',
 'The city name in that string is "ŒEdenburg"'

In [ ]:
question_italian = """
Nella seguente stringa, che contiene potenziali errori di battitura, sto cercando il nome di una città.
So che la città si trova in Italia, quindi si prega di indicare una città italiana. 
Forniscimi il nome italiano della città, nessun altro output:"
"""

In [148]:
data.to_csv('../data/1867_italy_chatgpt.csv', index=False)

In [16]:
responses[3243]

IndexError: list index out of range

In [30]:
flattened_list = responses + responses_from_3243

In [34]:
data['location_chatgpt'] = flattened_list

In [35]:
data.to_csv('../data/1867_austria_chatgpt.csv', index=False)